In [1]:
import time
import glob as glob
from zipfile import ZipFile
from bs4 import BeautifulSoup
import dateutil.parser as dparser
from datetime import datetime
import pandas as pd
import multiprocessing as mp
import numpy as np
import io
import json
import sys
import re

In [2]:
# root_dir = '/media/shane/cloud-availability-sacheen-2020-05-11/cloud-google-apps' # one directory
# root_dir = '/media/shane/cloud-availability-sacheen-2020-05-11/cloud-google-apps/2020/20204' #subset of above
root_dir = '/media/shane/cloud-availability-sacheen-2020-05-11/cloud-google-apps*' # all directories

In [3]:
def read_file(file):
    with ZipFile(file, "r") as z:
        for filename in z.namelist(): 
            if 'json' in filename: 
                with z.open(filename) as f:
                    ret = []
                    f.seek(16) # skip some non-json garbage
                    data = f.read()
                    data = data.split(b');')[0]
                    d = json.loads(data) 
                    for v in d.values():
                        ret.append([v])
                    return ret

In [4]:
files = glob.glob(root_dir + '/**/*.zip', recursive=True)
pool = mp.Pool(mp.cpu_count())
start_time = time.time()
try:
    results = pool.map(read_file, [file for file in files])
except:
    print("Unexpected error:", sys.exc_info()[0])
    raise
pool.close()
end_time = time.time()

In [5]:
unique = set()
counter = 0
try:
    for i in range (0, len(results)):
        for j in range(0, len(results[i][1][0])):
            counter += 1
            unique.add(str(results[i][1][0][j]).replace("\'",''))
except:
    pass

In [6]:
print(counter, len(unique))

54493 549


In [7]:
# print(unique.pop())

In [21]:
df_unique = pd.DataFrame(columns=['service', 'outageId', 'time', 'pst', 'message', 'type', 'resolved', 'premier'])
for entry in unique:
    arr = []
    arr.append(entry.split(':')[1].split(',')[0].strip())
    arr.append(entry.split(':')[2].split(',')[0].strip())
    arr.append(entry.split(':')[3].split(',')[0].strip())
    arr.append(entry.split('message:')[0].split('pst: ')[1].split(',')[:-1])
    arr.append(entry.split('type:')[0].split('message:')[1].split(',')[:-1])
    arr.append(entry.split('resolved:')[0].split('type:')[1].split(',')[:-1])
    arr.append(entry.split('premier:')[0].split('resolved:')[1].split(',')[:-1])
    arr.append(entry.split(':')[-1].split(',')[0].strip().replace('}','').replace('[',''))
    df_unique.loc[len(df_unique)] = arr

In [22]:
df_unique

,service,outageId,time,pst,message,type,resolved,premier
0,22,9b03916dfbdc9710283f467403264d16,1572232860000,"[October 27, 2019 8:21:00 PM PDT]",[ We are continuing to investigate this issue....,[ 1],[ True],False
1,0,51b20fb9ba3b898c8236936fca17fc8f,1576748940000,"[December 19, 2019 1:49:00 AM PST]",[ Our team is continuing to investigate this i...,[ 1],[ True],False
2,4,fcc552c5af396475ac90ebb7278829c3,1579723140000,"[January 22, 2020 11:59:00 AM PST]",[ The problem with Google Drive should be reso...,[ 3],[ True],Please have affected users re-authenticate to ...
3,1,036ce0c8fec1393dabdec4e39bed9437,1586361480000,"[April 8, 2020 8:58:00 AM PDT]",[ The problem with Gmail should be resolved. W...,[ 3],[ True],False
4,1,74a7efa47b9de665d02699bbe9dd11fc,1559518200000,"[June 2, 2019 4:30:00 PM PDT]",[ The problem with Gmail should be resolved. W...,[ 3],[ True],False
...,...,...,...,...,...,...,...,...
544,23,74a7efa47b9de665d02699bbe9dd11fc,1559505060000,"[June 2, 2019 12:51:00 PM PDT]",[ We are continuing to investigate this issue....,[ 2],[ True],False
545,0,4ef64f90d3c93afd1379f0c02be9803d,1586394480000,"[April 8, 2020 6:08:00 PM PDT]",[ Our team is continuing to investigate this i...,[ 1],[ False],Work continues to resolve the issue for all re...
546,43,74a7efa47b9de665d02699bbe9dd11fc,1559518200000,"[June 2, 2019 4:30:00 PM PDT]",[ The problem with Google Keep should be resol...,[ 3],[ True],False
547,37,4c6bd4230eac0ebd373c300b0846601b,1506373800000,"[September 25, 2017 2:10:00 PM PDT]",[ The problem with Google Vault should be reso...,[ 3],[ True],False


In [23]:
print(end_time - start_time)

0.22545647621154785


In [24]:
df_unique.to_csv(r'/home/shane/Documents/thesis/output/parsed/final/cloud-google-apps.csv', index=False)